# **!Pip and Import**

In [ ]:
!pip install adversarial-robustness-toolbox

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 13.3 MB/s eta 0:00:00


In [ ]:
!pip install tabulate

In [ ]:
!pip install openpyxl

In [ ]:
!pip show torch torchvision sympy

Name: torch
Version: 2.5.1+cu121
Summary: Tensors and Dynamic neural networks in Python with strong GPU acceleration
Home-page: https://pytorch.org/
Author: PyTorch Team
Author-email: packages@pytorch.org
License: BSD-3-Clause
Location: /usr/local/lib/python3.10/dist-packages
Requires: filelock, fsspec, jinja2, networkx, sympy, typing-extensions
Required-by: accelerate, fastai, peft, sentence-transformers, timm, torchaudio, torchvision
---
Name: torchvision
Version: 0.20.1+cu121
Summary: image and video datasets and models for torch deep learning
Home-page: https://github.com/pytorch/vision
Author: PyTorch Core Team
Author-email: soumith@pytorch.org
License: BSD
Location: /usr/local/lib/python3.10/dist-packages
Requires: numpy, pillow, torch
Required-by: fastai, timm
---
Name: sympy
Version: 1.13.1
Summary: Computer algebra system (CAS) in Python
Home-page: https://sympy.org
Author: SymPy development team
Author-email: sympy@googlegroups.com
License: BSD
Location: /usr/local/lib/python

In [ ]:
!pip install --upgrade torch torchvision torchaudio

In [ ]:
!pip install --upgrade sympy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 37.1 MB/s eta 0:00:00
  Attempting uninstall: sympy
    Found existing installation: sympy 1.13.1
    Uninstalling sympy-1.13.1:
      Successfully uninstalled sympy-1.13.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.5.1+cu121 requires sympy==1.13.1; python_version >= "3.9", but you have sympy 1.13.3 which is incompatible.


In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import torch.nn.functional as F
import torch.nn.init as init



In [ ]:
# Check if GPU is available, otherwise use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


# **Loading IoTID20**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Load data
data = pd.read_csv("/content/drive/MyDrive/KnowledgeDistillation/iotid20.csv")
data

,Src_Port,Dst_Port,Protocol,Flow_Duration,Tot_Fwd_Pkts,Tot_Bwd_Pkts,TotLen_Fwd_Pkts,TotLen_Bwd_Pkts,Fwd_Pkt_Len_Max,Fwd_Pkt_Len_Min,...,Fwd_Seg_Size_Min,Active_Mean,Active_Std,Active_Max,Active_Min,Idle_Mean,Idle_Std,Idle_Max,Idle_Min,Target
0,53182,9020,6,121,1,1,0.0,0.0,0.0,0.0,...,0,0.00,0.0,0.0,0.0,121.000000,0.000000,121.0,121.0,MITM ARP Spoofing
1,52779,443,6,146,2,1,0.0,0.0,0.0,0.0,...,0,0.00,0.0,0.0,0.0,73.000000,0.000000,73.0,73.0,MITM ARP Spoofing
2,443,33880,6,85,1,1,37.0,447.0,37.0,37.0,...,0,0.00,0.0,0.0,0.0,85.000000,0.000000,85.0,85.0,MITM ARP Spoofing
3,61861,443,6,147,1,1,151.0,366.0,151.0,151.0,...,0,0.00,0.0,0.0,0.0,147.000000,0.000000,147.0,147.0,MITM ARP Spoofing
4,52779,443,6,74,0,2,0.0,2896.0,0.0,0.0,...,0,0.00,0.0,0.0,0.0,74.000000,0.000000,74.0,74.0,MITM ARP Spoofing
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261533,60173,8899,17,31,4,1,128.0,32.0,32.0,32.0,...,0,0.00,0.0,0.0,0.0,7.750000,4.349329,12.0,4.0,Mirai
261534,48784,443,6,160,2,1,0.0,0.0,0.0,0.0,...,0,0.00,0.0,0.0,0.0,80.000000,5.656854,84.0,76.0,Mirai
261535,60098,8899,17,298,3,1,96.0,32.0,32.0,32.0,...,0,0.00,0.0,0.0,0.0,99.333333,157.366875,281.0,5.0,Mirai
261536,64783,9988,17,34,14,1,448.0,32.0,32.0,32.0,...,0,1.25,0.5,2.0,1.0,3.625000,2.774244,9.0,2.0,Mirai


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 261538 entries, 0 to 261537
Data columns (total 80 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Src_Port           261538 non-null  int64  
 1   Dst_Port           261538 non-null  int64  
 2   Protocol           261538 non-null  int64  
 3   Flow_Duration      261538 non-null  int64  
 4   Tot_Fwd_Pkts       261538 non-null  int64  
 5   Tot_Bwd_Pkts       261538 non-null  int64  
 6   TotLen_Fwd_Pkts    261538 non-null  float64
 7   TotLen_Bwd_Pkts    261538 non-null  float64
 8   Fwd_Pkt_Len_Max    261538 non-null  float64
 9   Fwd_Pkt_Len_Min    261538 non-null  float64
 10  Fwd_Pkt_Len_Mean   261538 non-null  float64
 11  Fwd_Pkt_Len_Std    261538 non-null  float64
 12  Bwd_Pkt_Len_Max    261538 non-null  float64
 13  Bwd_Pkt_Len_Min    261538 non-null  float64
 14  Bwd_Pkt_Len_Mean   261538 non-null  float64
 15  Bwd_Pkt_Len_Std    261538 non-null  float64
 16  Fl

In [ ]:
# Handle invalid values (infinity or too large)
data = data.replace([np.inf, -np.inf], np.nan)
data = data.dropna()
data = data.reset_index()

In [ ]:
# Remove unnecessary columns
data.drop(["Src_Port","Dst_Port","Protocol"],axis=1,inplace=True)

In [ ]:
data

,index,Flow_Duration,Tot_Fwd_Pkts,Tot_Bwd_Pkts,TotLen_Fwd_Pkts,TotLen_Bwd_Pkts,Fwd_Pkt_Len_Max,Fwd_Pkt_Len_Min,Fwd_Pkt_Len_Mean,Fwd_Pkt_Len_Std,...,Fwd_Seg_Size_Min,Active_Mean,Active_Std,Active_Max,Active_Min,Idle_Mean,Idle_Std,Idle_Max,Idle_Min,Target
0,0,121,1,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0.00,0.0,0.0,0.0,121.000000,0.000000,121.0,121.0,MITM ARP Spoofing
1,1,146,2,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0.00,0.0,0.0,0.0,73.000000,0.000000,73.0,73.0,MITM ARP Spoofing
2,2,85,1,1,37.0,447.0,37.0,37.0,37.0,0.0,...,0,0.00,0.0,0.0,0.0,85.000000,0.000000,85.0,85.0,MITM ARP Spoofing
3,3,147,1,1,151.0,366.0,151.0,151.0,151.0,0.0,...,0,0.00,0.0,0.0,0.0,147.000000,0.000000,147.0,147.0,MITM ARP Spoofing
4,4,74,0,2,0.0,2896.0,0.0,0.0,0.0,0.0,...,0,0.00,0.0,0.0,0.0,74.000000,0.000000,74.0,74.0,MITM ARP Spoofing
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261459,261533,31,4,1,128.0,32.0,32.0,32.0,32.0,0.0,...,0,0.00,0.0,0.0,0.0,7.750000,4.349329,12.0,4.0,Mirai
261460,261534,160,2,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0.00,0.0,0.0,0.0,80.000000,5.656854,84.0,76.0,Mirai
261461,261535,298,3,1,96.0,32.0,32.0,32.0,32.0,0.0,...,0,0.00,0.0,0.0,0.0,99.333333,157.366875,281.0,5.0,Mirai
261462,261536,34,14,1,448.0,32.0,32.0,32.0,32.0,0.0,...,0,1.25,0.5,2.0,1.0,3.625000,2.774244,9.0,2.0,Mirai


In [ ]:
data['Target'].value_counts()

,count
Target,
Mirai,143254
DoS,59362
Normal,27530
Scan,16274
MITM ARP Spoofing,15044


In [ ]:
# Separate features and labels
X = data.iloc[:,:-1]
y = data.iloc[:, -1:]

In [ ]:
X

,index,Flow_Duration,Tot_Fwd_Pkts,Tot_Bwd_Pkts,TotLen_Fwd_Pkts,TotLen_Bwd_Pkts,Fwd_Pkt_Len_Max,Fwd_Pkt_Len_Min,Fwd_Pkt_Len_Mean,Fwd_Pkt_Len_Std,...,Fwd_Act_Data_Pkts,Fwd_Seg_Size_Min,Active_Mean,Active_Std,Active_Max,Active_Min,Idle_Mean,Idle_Std,Idle_Max,Idle_Min
0,0,121,1,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0.00,0.0,0.0,0.0,121.000000,0.000000,121.0,121.0
1,1,146,2,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0.00,0.0,0.0,0.0,73.000000,0.000000,73.0,73.0
2,2,85,1,1,37.0,447.0,37.0,37.0,37.0,0.0,...,1,0,0.00,0.0,0.0,0.0,85.000000,0.000000,85.0,85.0
3,3,147,1,1,151.0,366.0,151.0,151.0,151.0,0.0,...,1,0,0.00,0.0,0.0,0.0,147.000000,0.000000,147.0,147.0
4,4,74,0,2,0.0,2896.0,0.0,0.0,0.0,0.0,...,0,0,0.00,0.0,0.0,0.0,74.000000,0.000000,74.0,74.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261459,261533,31,4,1,128.0,32.0,32.0,32.0,32.0,0.0,...,4,0,0.00,0.0,0.0,0.0,7.750000,4.349329,12.0,4.0
261460,261534,160,2,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0.00,0.0,0.0,0.0,80.000000,5.656854,84.0,76.0
261461,261535,298,3,1,96.0,32.0,32.0,32.0,32.0,0.0,...,3,0,0.00,0.0,0.0,0.0,99.333333,157.366875,281.0,5.0
261462,261536,34,14,1,448.0,32.0,32.0,32.0,32.0,0.0,...,14,0,1.25,0.5,2.0,1.0,3.625000,2.774244,9.0,2.0


In [ ]:
y

,Target
0,MITM ARP Spoofing
1,MITM ARP Spoofing
2,MITM ARP Spoofing
3,MITM ARP Spoofing
4,MITM ARP Spoofing
...,...
261459,Mirai
261460,Mirai
261461,Mirai
261462,Mirai


In [ ]:
# Encode labels
from sklearn.preprocessing import OneHotEncoder

# Initialize OneHotEncoder
onehot_encoder = OneHotEncoder(sparse_output=False)

# Perform One-Hot Encoding on the 'Target' column
y = onehot_encoder.fit_transform(y)


In [ ]:
y

array([[0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       ...,
       [0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.]])

In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
X_train.shape

(209171, 77)

In [ ]:
output_size = y.shape[1]
output_size

5

In [ ]:
# Normalize the data
from sklearn.preprocessing import MinMaxScaler
min_max_scaler = MinMaxScaler()
X_train = min_max_scaler.fit_transform(X_train)
X_test = min_max_scaler.transform(X_test)

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convert features to tensor
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)

# Convert labels from One-Hot Encoding back to single labels
labels_train = y_train.argmax(axis=1)
labels_test = y_test.argmax(axis=1)

# Convert labels to tensor
labels_train_tensor = torch.tensor(labels_train, dtype=torch.long)
labels_test_tensor = torch.tensor(labels_test, dtype=torch.long)

# Create Dataset and DataLoader
train_data = TensorDataset(X_train_tensor, labels_train_tensor)
test_data = TensorDataset(X_test_tensor, labels_test_tensor)

batch_size = 128
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

# **Define MLP Model**

In [ ]:
# Define the TeacherMLP model
class TeacherMLP(nn.Module):
    def __init__(self, input_size, output_size):
        super(TeacherMLP, self).__init__()
        self.fc1 = nn.Linear(input_size, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, output_size)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.6)
        self.batch_norm1 = nn.BatchNorm1d(512)
        self.batch_norm2 = nn.BatchNorm1d(256)
        nn.init.kaiming_normal_(self.fc1.weight, mode='fan_in', nonlinearity='relu')
        nn.init.kaiming_normal_(self.fc2.weight, mode='fan_in', nonlinearity='relu')
        nn.init.xavier_normal_(self.fc3.weight)

    def forward(self, x):
        x = self.fc1(x)
        x = self.batch_norm1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.batch_norm2(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc3(x)
        x = F.softmax(x, dim=1)
        return x

In [ ]:
# Define the StudentMLP model
class StudentMLP(nn.Module):
    def __init__(self, input_size, output_size):
        super(StudentMLP, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, output_size)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.6)
        self.batch_norm1 = nn.BatchNorm1d(64)
        self.batch_norm2 = nn.BatchNorm1d(32)
        nn.init.kaiming_normal_(self.fc1.weight, mode='fan_in', nonlinearity='relu')
        nn.init.kaiming_normal_(self.fc2.weight, mode='fan_in', nonlinearity='relu')
        nn.init.xavier_normal_(self.fc3.weight)

    def forward(self, x):
        x = self.fc1(x)
        x = self.batch_norm1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.batch_norm2(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc3(x)
        x = F.softmax(x, dim=1)
        return x


# **Training - Evaluation - Testing**

In [ ]:
import torch.optim as optim
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Training function
def train(model, train_loader, test_loader, epochs, learning_rate, device, patience=5):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=3, factor=0.5)
    model.to(device)
    model.train()

    best_test_loss = float('inf')
    patience_counter = 0

    for epoch in range(epochs):
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)

            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        avg_train_loss = running_loss / len(train_loader)
        avg_test_loss = evaluate(model, test_loader, criterion, device)

        print(f"Epoch {epoch+1}/{epochs}, Train Loss: {avg_train_loss:.4f}, Test Loss: {avg_test_loss:.4f}")

        # Step the scheduler
        scheduler.step(avg_test_loss)

        # Early Stopping
        if avg_test_loss < best_test_loss:
            best_test_loss = avg_test_loss
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f"Early stopping at epoch {epoch+1}")
                break

# Evaluation function
def evaluate(model, data_loader, criterion, device):
    model.eval()
    loss = 0.0

    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss += criterion(outputs, labels).item()

    avg_loss = loss / len(data_loader)
    model.train()
    return avg_loss

# Testing function
def test(model, data_loader, device):
    model.to(device)
    model.eval()

    predictions = []
    true_labels = []

    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)

            predictions.extend(predicted.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())

    accuracy = 100 * accuracy_score(true_labels, predictions)
    precision = 100 * precision_score(true_labels, predictions, average='weighted', zero_division=1)
    recall = 100 * recall_score(true_labels, predictions, average='weighted', zero_division=1)
    f1 = 100 * f1_score(true_labels, predictions, average='weighted', zero_division=1)

    return accuracy, precision, recall, f1


# **Cross-Entropy**

In [ ]:
# Set random seed for reproducibility
torch.manual_seed(42)

# Initialize the Teacher MLP model
teacher_mlp = TeacherMLP(input_size=X_train.shape[1], output_size=output_size).to(device)

# Train the Teacher MLP model
train(teacher_mlp, train_loader, test_loader, epochs=10, learning_rate=0.001, device=device)

Epoch 1/10, Train Loss: 0.9718, Test Loss: 0.9248
Epoch 2/10, Train Loss: 0.9362, Test Loss: 0.9206
Epoch 3/10, Train Loss: 0.9315, Test Loss: 0.9176
Epoch 4/10, Train Loss: 0.9300, Test Loss: 0.9200
Epoch 5/10, Train Loss: 0.9288, Test Loss: 0.9176
Epoch 6/10, Train Loss: 0.9281, Test Loss: 0.9182
Epoch 7/10, Train Loss: 0.9268, Test Loss: 0.9168
Epoch 8/10, Train Loss: 0.9263, Test Loss: 0.9176
Epoch 9/10, Train Loss: 0.9262, Test Loss: 0.9176
Epoch 10/10, Train Loss: 0.9264, Test Loss: 0.9158


In [ ]:
# Evaluate the Teacher MLP model on the training and test sets
train_metrics_teacher_mlp = test(teacher_mlp, train_loader, device)
test_metrics_teacher_mlp = test(teacher_mlp, test_loader, device)

print(f"Teacher Model - Train Accuracy: {train_metrics_teacher_mlp[0]:.2f}%, Test Accuracy: {test_metrics_teacher_mlp[0]:.2f}%")
print(f"Teacher Model - Train Precision: {train_metrics_teacher_mlp[1]:.2f}%, Test Precision: {test_metrics_teacher_mlp[1]:.2f}%")
print(f"Teacher Model - Train Recall: {train_metrics_teacher_mlp[2]:.2f}%, Test Recall: {test_metrics_teacher_mlp[2]:.2f}%")
print(f"Teacher Model - Train F1-score: {train_metrics_teacher_mlp[3]:.2f}%, Test F1-score: {test_metrics_teacher_mlp[3]:.2f}%")

Teacher Model - Train Accuracy: 98.85%, Test Accuracy: 98.90%
Teacher Model - Train Precision: 98.88%, Test Precision: 98.93%
Teacher Model - Train Recall: 98.85%, Test Recall: 98.90%
Teacher Model - Train F1-score: 98.86%, Test F1-score: 98.91%


In [ ]:
# Set seed for reproducibility
torch.manual_seed(42)

# Initialize the Student MLP model
student_mlp = StudentMLP(input_size=X_train.shape[1], output_size=output_size).to(device)

# Train the Student MLP model [ initial lightweight model ]
train(student_mlp, train_loader, test_loader, epochs=10, learning_rate=0.001, device=device)

Epoch 1/10, Train Loss: 1.0793, Test Loss: 0.9876
Epoch 2/10, Train Loss: 1.0014, Test Loss: 0.9395
Epoch 3/10, Train Loss: 0.9735, Test Loss: 0.9294
Epoch 4/10, Train Loss: 0.9629, Test Loss: 0.9245
Epoch 5/10, Train Loss: 0.9561, Test Loss: 0.9234
Epoch 6/10, Train Loss: 0.9526, Test Loss: 0.9232
Epoch 7/10, Train Loss: 0.9510, Test Loss: 0.9244
Epoch 8/10, Train Loss: 0.9478, Test Loss: 0.9195
Epoch 9/10, Train Loss: 0.9474, Test Loss: 0.9226
Epoch 10/10, Train Loss: 0.9454, Test Loss: 0.9175


In [ ]:
# Evaluate the model on the training and test sets
train_metrics_student_mlp = test(student_mlp, train_loader, device)
test_metrics_student_mlp = test(student_mlp, test_loader, device)

print(f"Student Model - Train Accuracy: {train_metrics_student_mlp[0]:.2f}%, Test Accuracy: {test_metrics_student_mlp[0]:.2f}%")
print(f"Student Model - Train Precision: {train_metrics_student_mlp[1]:.2f}%, Test Precision: {test_metrics_student_mlp[1]:.2f}%")
print(f"Student Model - Train Recall: {train_metrics_student_mlp[2]:.2f}%, Test Recall: {test_metrics_student_mlp[2]:.2f}%")
print(f"Student Model - Train F1-score: {train_metrics_student_mlp[3]:.2f}%, Test F1-score: {test_metrics_student_mlp[3]:.2f}%")


Student Model - Train Accuracy: 98.80%, Test Accuracy: 98.77%
Student Model - Train Precision: 98.85%, Test Precision: 98.82%
Student Model - Train Recall: 98.80%, Test Recall: 98.77%
Student Model - Train F1-score: 98.81%, Test F1-score: 98.78%


In [ ]:
from tabulate import tabulate

# Data
data = [
    ["Teacher MLP Metrics", f"{test_metrics_teacher_mlp[0]:.2f}%", f"{test_metrics_teacher_mlp[1]:.2f}%", f"{test_metrics_teacher_mlp[2]:.2f}%", f"{test_metrics_teacher_mlp[3]:.2f}%"],
    ["Student MLP Metrics", f"{test_metrics_student_mlp[0]:.2f}%", f"{test_metrics_student_mlp[1]:.2f}%", f"{test_metrics_student_mlp[2]:.2f}%", f"{test_metrics_student_mlp[3]:.2f}%"],
]

# Column headers
headers = ["Metrics", "Accuracy", "Precision", "Recall", "F1-score"]

# Display table
print(tabulate(data, headers=headers, tablefmt="grid"))


+---------------------+------------+-------------+----------+------------+
| Metrics             | Accuracy   | Precision   | Recall   | F1-score   |
+=====================+============+=============+==========+============+
| Teacher MLP Metrics | 98.90%     | 98.93%      | 98.90%   | 98.91%     |
+---------------------+------------+-------------+----------+------------+
| Student MLP Metrics | 98.77%     | 98.82%      | 98.77%   | 98.78%     |
+---------------------+------------+-------------+----------+------------+


In [ ]:
# Count the number of parameters in the TeacherMLP model
total_params_deep = "{:,}".format(sum(p.numel() for p in teacher_mlp.parameters()))
print(f"TeacherMLP parameters: {total_params_deep}")

# Count the number of parameters in the StudentMLP model
total_params_light = "{:,}".format(sum(p.numel() for p in student_mlp.parameters()))
print(f"StudentMLP parameters: {total_params_light}")

TeacherMLP parameters: 174,085
StudentMLP parameters: 7,429


# **Knowledge Distillation**

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score

def train_knowledge_distillation(teacher, student, train_loader, epochs, learning_rate, T, soft_target_loss_weight, ce_loss_weight, device):
    ce_loss = nn.CrossEntropyLoss()
    optimizer = optim.Adam(student.parameters(), lr=learning_rate)

    teacher.eval()  # Set teacher to evaluation mode
    student.train() # Set student to training mode

    for epoch in range(epochs):
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()

            # Forward pass with teacher model
            with torch.no_grad():
                teacher_logits = teacher(inputs)

            # Forward pass with student model
            student_logits = student(inputs)

            # Compute soft targets
            soft_targets = F.softmax(teacher_logits / T, dim=-1)
            soft_prob = F.log_softmax(student_logits / T, dim=-1)

            # KL Divergence Loss
            soft_targets_loss = F.kl_div(soft_prob, soft_targets, reduction='batchmean')

            # Compute loss for actual labels
            label_loss = ce_loss(student_logits, labels)

            # Total loss
            loss = soft_target_loss_weight * soft_targets_loss + ce_loss_weight * label_loss

            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss / len(train_loader)}")



In [ ]:
# Set random seed for reproducibility
torch.manual_seed(42)

# Initialize a new [ MLP ] model for the lightweight network [--> New Student MLP model]
new_student_mlp = StudentMLP(input_size=X_train.shape[1], output_size=output_size).to(device)

In [ ]:
# Print the norm of the first layer of the original lightweight model
print("Norm of 1st layer of student_mlp:", torch.norm(student_mlp.fc1.weight).item())

# Print the norm of the first layer of the new lightweight model
print("Norm of 1st layer of new_student_mlp:", torch.norm(new_student_mlp.fc1.weight).item())

Norm of 1st layer of student_mlp: 13.240311622619629
Norm of 1st layer of new_student_mlp: 11.105559349060059


In [ ]:
# Apply train_knowledge_distillation with temperature and weights
train_knowledge_distillation(teacher=teacher_mlp,
                             student=new_student_mlp,
                             train_loader=train_loader,
                             epochs=10,
                             learning_rate=0.001,
                             T=2,
                             soft_target_loss_weight=0.25,
                             ce_loss_weight=0.75,
                             device=device)

# Evaluate the new student model
test_metrics_kd_student_mlp = test(new_student_mlp, test_loader, device)

Epoch 1/10, Loss: 0.8115567583556569
Epoch 2/10, Loss: 0.7518114130431359
Epoch 3/10, Loss: 0.7313881142059233
Epoch 4/10, Loss: 0.7222771949359765
Epoch 5/10, Loss: 0.716941623184659
Epoch 6/10, Loss: 0.7146480313499404
Epoch 7/10, Loss: 0.7128341561792823
Epoch 8/10, Loss: 0.7110766928859442
Epoch 9/10, Loss: 0.7098777026576004
Epoch 10/10, Loss: 0.7090111296840399


In [ ]:
from tabulate import tabulate

# Data
data = [
    ["Teacher MLP Metrics", f"{test_metrics_teacher_mlp[0]:.2f}%", f"{test_metrics_teacher_mlp[1]:.2f}%", f"{test_metrics_teacher_mlp[2]:.2f}%", f"{test_metrics_teacher_mlp[3]:.2f}%"],
    ["Student MLP Metrics", f"{test_metrics_student_mlp[0]:.2f}%", f"{test_metrics_student_mlp[1]:.2f}%", f"{test_metrics_student_mlp[2]:.2f}%", f"{test_metrics_student_mlp[3]:.2f}%"],
    ["KD Student MLP Metrics", f"{test_metrics_kd_student_mlp[0]:.2f}%", f"{test_metrics_kd_student_mlp[1]:.2f}%", f"{test_metrics_kd_student_mlp[2]:.2f}%", f"{test_metrics_kd_student_mlp[3]:.2f}%"],
]

# Column headers
headers = ["Metrics", "Accuracy", "Precision", "Recall", "F1-score"]

# Display the table
print(tabulate(data, headers=headers, tablefmt="grid"))


+------------------------+------------+-------------+----------+------------+
| Metrics                | Accuracy   | Precision   | Recall   | F1-score   |
+========================+============+=============+==========+============+
| Teacher MLP Metrics    | 98.90%     | 98.93%      | 98.90%   | 98.91%     |
+------------------------+------------+-------------+----------+------------+
| Student MLP Metrics    | 98.77%     | 98.82%      | 98.77%   | 98.78%     |
+------------------------+------------+-------------+----------+------------+
| KD Student MLP Metrics | 98.81%     | 98.84%      | 98.81%   | 98.82%     |
+------------------------+------------+-------------+----------+------------+


# **Adversarial Attacks**

## **Preparation**

In [ ]:
import art
print(art.__version__)

1.18.2


In [ ]:
print(f"Using device: {device}")

Using device: cpu


In [ ]:
print(f"Teacher model device: {next(teacher_mlp.parameters()).device}")
print(f"Student model device: {next(student_mlp.parameters()).device}")
print(f"New Student model device: {next(new_student_mlp.parameters()).device}")

Teacher model device: cpu
Student model device: cpu
New Student model device: cpu


In [ ]:
print(f"X_train_tensor device: {X_train_tensor.device}")
print(f"X_test_tensor device: {X_test_tensor.device}")
print(f"Labels_train_tensor device: {labels_train_tensor.device}")
print(f"Labels_test_tensor device: {labels_test_tensor.device}")

X_train_tensor device: cpu
X_test_tensor device: cpu
Labels_train_tensor device: cpu
Labels_test_tensor device: cpu


In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from art.estimators.classification.pytorch import PyTorchClassifier
import torch.nn.functional as F
from art.attacks.evasion import FastGradientMethod
from art.estimators.classification import PyTorchClassifier
from art.utils import load_mnist

# Define the loss criterion
criterion = nn.CrossEntropyLoss()

# Find the minimum value in the data and assign it to the variable min_
min_ = np.min(X_train)

# Find the maximum value in the data and assign it to the variable max_
max_ = np.max(X_train)

# Find the number of unique classes
unique_classes = torch.unique(labels_train_tensor)
nb_classes = len(unique_classes)

In [ ]:
# Choose the optimization algorithm and its hyperparameters, e.g., Adam optimizer
optimizer_teacher_mlp = optim.Adam(teacher_mlp.parameters(), lr=0.001)
optimizer_student_mlp = optim.Adam(student_mlp.parameters(), lr=0.001)
optimizer_new_student_mlp = optim.Adam(new_student_mlp.parameters(), lr=0.001)

# Wrap teacher_mlp, student_mlp, and new_student_mlp with PyTorchClassifier
teacher_mlp_classifier = PyTorchClassifier(
    model=teacher_mlp,
    loss=criterion,
    optimizer=optimizer_teacher_mlp,
    input_shape=(1, X_train.shape[1]),
    nb_classes=nb_classes,
    clip_values=(min_, max_),
)
student_mlp_classifier = PyTorchClassifier(
    model=student_mlp,
    loss=criterion,
    optimizer=optimizer_student_mlp,
    input_shape=(1, X_train.shape[1]),
    nb_classes=nb_classes,
    clip_values=(min_, max_)
)
new_student_mlp_classifier = PyTorchClassifier(
    model=new_student_mlp,
    loss=criterion,
    optimizer=optimizer_new_student_mlp,
    input_shape=(1, X_train.shape[1]),
    nb_classes=nb_classes,
    clip_values=(min_, max_)
)

In [ ]:
# Check the device of the teacher_mlp model inside the teacher_classifier
device = next(teacher_mlp_classifier.model.parameters()).device
print(f'Teacher model is on device: {device}')

# Check the device of the student_mlp model inside the student_classifier
device = next(student_mlp_classifier.model.parameters()).device
print(f'Student model is on device: {device}')

# Check the device of the new_student_mlp model inside the new_student_classifier
device = next(new_student_mlp_classifier.model.parameters()).device
print(f'New Student model is on device: {device}')

Teacher model is on device: cpu
Student model is on device: cpu
New Student model is on device: cpu


## **ART.Attacks.Evasion**

***Lấy một vài mẫu để test***

In [ ]:
# Use numpy() to convert tensor to NumPy array
x_test_numpy = X_test_tensor.detach().cpu().numpy().astype(np.float32)
print(f"x_test_numpy dtype: {x_test_numpy.dtype}")

x_test_numpy dtype: float32


In [ ]:
x_test_numpy.shape

(52293, 77)

In [ ]:
# Shorten the data
total_samples = x_test_numpy.shape[0]
percentage = 0.05  # Select 5% of the original data
num_samples = int(total_samples * percentage)
num_samples

2614

In [ ]:
# Select a small portion of the test data
x_test_numpy = x_test_numpy[:num_samples]
x_test_numpy.shape

(2614, 77)

In [ ]:
labels_test.shape

(52293,)

In [ ]:
labels_test = labels_test[:num_samples]
labels_test

array([2, 2, 2, ..., 2, 0, 3])

In [ ]:
labels_test.shape

(2614,)

### **Fast Gradient Method (FGM) Attack**

In [ ]:
from art.attacks.evasion import FastGradientMethod

# Define parameters for Fast Gradient Method (FGM) Attack
fgm_params = {
    'norm': np.inf, # Use L∞ norm for the perturbation, focusing on maximizing individual pixel changes
    'eps': 0.3, # Maximum allowed perturbation per pixel, controlling overall noise level
    'eps_step': 0.1, # Maximum allowed perturbation per pixel, controlling overall noise level
    'targeted': False,  # Set to False for untargeted attack, aiming to mislead the model without a specific target
    'batch_size': 128 # Number of samples to attack in each batch, optimizing processing speed
}

# Use the wrapped estimator to create attackers and generate adversarial samples #
# Teacher MLP
teacher_mlp_fgm_attacker = FastGradientMethod(estimator=teacher_mlp_classifier, **fgm_params)

# Student MLP
student_mlp_fgm_attacker = FastGradientMethod(estimator=student_mlp_classifier, **fgm_params)

# New Student MLP
new_student_mlp_fgm_attacker = FastGradientMethod(estimator=new_student_mlp_classifier, **fgm_params)

# Generate adversarial samples from the teacher model
x_test_fgm_teacher_mlp = teacher_mlp_fgm_attacker.generate(x_test_numpy)

# Generate adversarial samples from the original student model
x_test_fgm_student_mlp = student_mlp_fgm_attacker.generate(x_test_numpy)

# Generate adversarial samples from the new student model
x_test_fgm_new_student_mlp = new_student_mlp_fgm_attacker.generate(x_test_numpy)

In [ ]:
# Convert NumPy arrays to Tensor #
# Teacher MLP
x_test_fgm_teacher_mlp_tensor = torch.tensor(x_test_fgm_teacher_mlp, dtype=torch.float32)

# Student MLP
x_test_fgm_student_mlp_tensor = torch.tensor(x_test_fgm_student_mlp, dtype=torch.float32)

# New Student MLP
x_test_fgm_new_student_mlp_tensor = torch.tensor(x_test_fgm_new_student_mlp, dtype=torch.float32)

# Label (labels_test = pd.factorize(y_test)[0]) #
# Teacher & Student & New Student MLP
labels_test_fgm_all_model_tensor = torch.tensor(labels_test, dtype=torch.long)

# Create Dataset #
# Teacher MLP
x_test_fgm_teacher_mlp_data = TensorDataset(x_test_fgm_teacher_mlp_tensor, labels_test_fgm_all_model_tensor)

# Student MLP
x_test_fgm_student_mlp_data = TensorDataset(x_test_fgm_student_mlp_tensor, labels_test_fgm_all_model_tensor)

# New Student MLP
x_test_fgm_new_student_mlp_data = TensorDataset(x_test_fgm_new_student_mlp_tensor, labels_test_fgm_all_model_tensor)

# Create DataLoader for the adversarial data #
# Teacher MLP
x_test_fgm_teacher_mlp_loader = DataLoader(x_test_fgm_teacher_mlp_data, batch_size=128, shuffle=False)

# Student MLP
x_test_fgm_student_mlp_loader = DataLoader(x_test_fgm_student_mlp_data, batch_size=128, shuffle=False)

# New Student MLP
x_test_fgm_new_student_mlp_loader = DataLoader(x_test_fgm_new_student_mlp_data, batch_size=128, shuffle=False)

In [ ]:
# Metrics for Teacher after the FGM Attack
test_metrics_teacher_mlp_fgm = test(teacher_mlp, x_test_fgm_teacher_mlp_loader, device)

# Metrics for Student after the FGM Attack
test_metrics_student_mlp_fgm = test(student_mlp, x_test_fgm_student_mlp_loader, device)

# Metrics for Student (KD) after the FGM Attack
test_metrics_new_student_mlp_fgm = test(new_student_mlp, x_test_fgm_new_student_mlp_loader, device)

In [ ]:
from tabulate import tabulate

# Data
data = [
    ["Teacher MLP Metrics", f"{test_metrics_teacher_mlp[0]:.2f}%", f"{test_metrics_teacher_mlp[1]:.2f}%", f"{test_metrics_teacher_mlp[2]:.2f}%", f"{test_metrics_teacher_mlp[3]:.2f}%"],

    ["Student MLP Metrics", f"{test_metrics_student_mlp[0]:.2f}%", f"{test_metrics_student_mlp[1]:.2f}%", f"{test_metrics_student_mlp[2]:.2f}%", f"{test_metrics_student_mlp[3]:.2f}%"],

    ["KD Student MLP Metrics", f"{test_metrics_kd_student_mlp[0]:.2f}%", f"{test_metrics_kd_student_mlp[1]:.2f}%", f"{test_metrics_kd_student_mlp[2]:.2f}%", f"{test_metrics_kd_student_mlp[3]:.2f}%"],

    ["Teacher MLP Metrics (FGM)", f"{test_metrics_teacher_mlp_fgm[0]:.2f}%", f"{test_metrics_teacher_mlp_fgm[1]:.2f}%", f"{test_metrics_teacher_mlp_fgm[2]:.2f}%", f"{test_metrics_teacher_mlp_fgm[3]:.2f}%"],

    ["Student MLP Metrics (FGM)", f"{test_metrics_student_mlp_fgm[0]:.2f}%", f"{test_metrics_student_mlp_fgm[1]:.2f}%", f"{test_metrics_student_mlp_fgm[2]:.2f}%", f"{test_metrics_student_mlp_fgm[3]:.2f}%"],

    ["KD Student MLP Metrics (FGM)", f"{test_metrics_new_student_mlp_fgm[0]:.2f}%", f"{test_metrics_new_student_mlp_fgm[1]:.2f}%", f"{test_metrics_new_student_mlp_fgm[2]:.2f}%", f"{test_metrics_new_student_mlp_fgm[3]:.2f}%"],
]

# Column headers
headers = ["Metrics", "Accuracy", "Precision", "Recall", "F1-score"]

# Display table
print(tabulate(data, headers=headers, tablefmt="grid"))


+------------------------------+------------+-------------+----------+------------+
| Metrics                      | Accuracy   | Precision   | Recall   | F1-score   |
+==============================+============+=============+==========+============+
| Teacher MLP Metrics          | 98.90%     | 98.93%      | 98.90%   | 98.91%     |
+------------------------------+------------+-------------+----------+------------+
| Student MLP Metrics          | 98.77%     | 98.82%      | 98.77%   | 98.78%     |
+------------------------------+------------+-------------+----------+------------+
| KD Student MLP Metrics       | 98.81%     | 98.84%      | 98.81%   | 98.82%     |
+------------------------------+------------+-------------+----------+------------+
| Teacher MLP Metrics (FGM)    | 76.01%     | 78.91%      | 76.01%   | 77.31%     |
+------------------------------+------------+-------------+----------+------------+
| Student MLP Metrics (FGM)    | 78.96%     | 83.49%      | 78.96%   | 80.66

### **Projected Gradient Descent (PGD) Attack**

In [ ]:
from art.attacks.evasion import ProjectedGradientDescentPyTorch

# Define parameters for Projected Gradient Descent (PGD) Attack
pgd_params = {
    'norm': 2,  # Use L2 norm for the perturbation, allowing controlled, gradual changes across pixels
    'eps': 0.3, # Maximum allowed perturbation per sample, ensuring the adversarial sample remains close to the original
    'eps_step': 0.1,  # Step size in each gradient update, iteratively adding perturbation without exceeding eps
    'max_iter': 100,   # Maximum number of iterations per sample, controlling the depth of the attack process
    'batch_size': 128,  # Number of samples to attack in each batch, optimizing batch processing for faster computation
}

# Use the wrapped estimator to create attackers and generate adversarial samples #
# Teacher MLP
teacher_mlp_pgd_attacker = ProjectedGradientDescentPyTorch(estimator=teacher_mlp_classifier, **pgd_params)

# StudentMLP
student_mlp_pgd_attacker = ProjectedGradientDescentPyTorch(estimator=student_mlp_classifier, **pgd_params)

# New StudentMLP
new_student_mlp_pgd_attacker = ProjectedGradientDescentPyTorch(estimator=new_student_mlp_classifier, **pgd_params)

# Generate adversarial samples from the teacher model
x_test_pgd_teacher_mlp = teacher_mlp_pgd_attacker.generate(x_test_numpy)

# Generate adversarial samples from the original student model
x_test_pgd_student_mlp = student_mlp_pgd_attacker.generate(x_test_numpy)

# Generate adversarial samples from the new student model
x_test_pgd_new_student_mlp = new_student_mlp_pgd_attacker.generate(x_test_numpy)

PGD - Batches:   0%|          | 0/21 [00:00<?, ?it/s]

PGD - Batches:   0%|          | 0/21 [00:00<?, ?it/s]

PGD - Batches:   0%|          | 0/21 [00:00<?, ?it/s]

In [ ]:
# Convert NumPy arrays to Tensor #
# Teacher MLP
x_test_pgd_teacher_mlp_tensor = torch.tensor(x_test_pgd_teacher_mlp, dtype=torch.float32)

# Student MLP
x_test_pgd_student_mlp_tensor = torch.tensor(x_test_pgd_student_mlp, dtype=torch.float32)

# New Student MLP
x_test_pgd_new_student_mlp_tensor = torch.tensor(x_test_pgd_new_student_mlp, dtype=torch.float32)

# Label (labels_test = pd.factorize(y_test)[0]) #
# Teacher & Student & New Student MLP
labels_test_pgd_all_model_tensor = torch.tensor(labels_test, dtype=torch.long)

# Create Dataset #
# Teacher MLP
x_test_pgd_teacher_mlp_data = TensorDataset(x_test_pgd_teacher_mlp_tensor, labels_test_pgd_all_model_tensor)

# Student MLP
x_test_pgd_student_mlp_data = TensorDataset(x_test_pgd_student_mlp_tensor, labels_test_pgd_all_model_tensor)

# New Student MLP
x_test_pgd_new_student_mlp_data = TensorDataset(x_test_pgd_new_student_mlp_tensor, labels_test_pgd_all_model_tensor)

# Create DataLoader for the adversarial data #
# Teacher MLP
x_test_pgd_teacher_mlp_loader = DataLoader(x_test_pgd_teacher_mlp_data, batch_size=128, shuffle=False)

# Student MLP
x_test_pgd_student_mlp_loader = DataLoader(x_test_pgd_student_mlp_data, batch_size=128, shuffle=False)

# New Student MLP
x_test_pgd_new_student_mlp_loader = DataLoader(x_test_pgd_new_student_mlp_data, batch_size=128, shuffle=False)

In [ ]:
# Metrics for Teacher after the PGD Attack
test_metrics_teacher_mlp_pgd = test(teacher_mlp, x_test_pgd_teacher_mlp_loader, device)

# Metrics for Student after the PGD Attack
test_metrics_student_mlp_pgd = test(student_mlp, x_test_pgd_student_mlp_loader, device)

# Metrics for Student (KD) after the PGD Attack
test_metrics_new_student_mlp_pgd = test(new_student_mlp, x_test_pgd_new_student_mlp_loader, device)

In [ ]:
from tabulate import tabulate

# Data
data = [
    ["Teacher MLP Metrics", f"{test_metrics_teacher_mlp[0]:.2f}%", f"{test_metrics_teacher_mlp[1]:.2f}%", f"{test_metrics_teacher_mlp[2]:.2f}%", f"{test_metrics_teacher_mlp[3]:.2f}%"],

    ["Student MLP Metrics", f"{test_metrics_student_mlp[0]:.2f}%", f"{test_metrics_student_mlp[1]:.2f}%", f"{test_metrics_student_mlp[2]:.2f}%", f"{test_metrics_student_mlp[3]:.2f}%"],

    ["KD Student MLP Metrics", f"{test_metrics_kd_student_mlp[0]:.2f}%", f"{test_metrics_kd_student_mlp[1]:.2f}%", f"{test_metrics_kd_student_mlp[2]:.2f}%", f"{test_metrics_kd_student_mlp[3]:.2f}%"],

    ["Teacher MLP Metrics (PGD)", f"{test_metrics_teacher_mlp_pgd[0]:.2f}%", f"{test_metrics_teacher_mlp_pgd[1]:.2f}%", f"{test_metrics_teacher_mlp_pgd[2]:.2f}%", f"{test_metrics_teacher_mlp_pgd[3]:.2f}%"],

    ["Student MLP Metrics (PGD)", f"{test_metrics_student_mlp_pgd[0]:.2f}%", f"{test_metrics_student_mlp_pgd[1]:.2f}%", f"{test_metrics_student_mlp_pgd[2]:.2f}%", f"{test_metrics_student_mlp_pgd[3]:.2f}%"],

    ["KD Student MLP Metrics (PGD)", f"{test_metrics_new_student_mlp_pgd[0]:.2f}%", f"{test_metrics_new_student_mlp_pgd[1]:.2f}%", f"{test_metrics_new_student_mlp_pgd[2]:.2f}%", f"{test_metrics_new_student_mlp_pgd[3]:.2f}%"],
]

# Column headers
headers = ["Metrics", "Accuracy", "Precision", "Recall", "F1-score"]

# Display table
print(tabulate(data, headers=headers, tablefmt="grid"))


+------------------------------+------------+-------------+----------+------------+
| Metrics                      | Accuracy   | Precision   | Recall   | F1-score   |
+==============================+============+=============+==========+============+
| Teacher MLP Metrics          | 98.90%     | 98.93%      | 98.90%   | 98.91%     |
+------------------------------+------------+-------------+----------+------------+
| Student MLP Metrics          | 98.77%     | 98.82%      | 98.77%   | 98.78%     |
+------------------------------+------------+-------------+----------+------------+
| KD Student MLP Metrics       | 98.81%     | 98.84%      | 98.81%   | 98.82%     |
+------------------------------+------------+-------------+----------+------------+
| Teacher MLP Metrics (PGD)    | 86.34%     | 86.18%      | 86.34%   | 86.20%     |
+------------------------------+------------+-------------+----------+------------+
| Student MLP Metrics (PGD)    | 91.55%     | 93.10%      | 91.55%   | 92.06

### **DeepFool Attack**

In [ ]:
from art.attacks.evasion import DeepFool

# Define parameters for the DeepFool Attack
deepfool_params = {
    'epsilon': 1e-6,  # Small value to control attack precision, ensuring minimal distortion from the original sample
    'max_iter': 50,  # Maximum number of iterations per sample, determining the depth of the attack
    'nb_grads': 5,  # Number of gradients to use in each iteration, enhancing attack accuracy
    'batch_size': 128,  # Number of samples processed in each batch, improving computational efficiency
    'verbose': True,  # Enable detailed logging of the attack process for monitoring purposes
}

# Create attackers for each classifier
# Teacher MLP
teacher_mlp_deepfool_attacker = DeepFool(classifier=teacher_mlp_classifier, **deepfool_params)

# Student MLP
student_mlp_deepfool_attacker = DeepFool(classifier=student_mlp_classifier, **deepfool_params)

# New Student MLP
new_student_mlp_deepfool_attacker = DeepFool(classifier=new_student_mlp_classifier, **deepfool_params)

# Generate adversarial samples from the teacher model
x_test_deepfool_teacher_mlp = teacher_mlp_deepfool_attacker.generate(x_test_numpy)

# Generate adversarial samples from the original student model
x_test_deepfool_student_mlp = student_mlp_deepfool_attacker.generate(x_test_numpy)

# Generate adversarial samples from the new student model
x_test_deepfool_new_student_mlp = new_student_mlp_deepfool_attacker.generate(x_test_numpy)


DeepFool:   0%|          | 0/21 [00:00<?, ?it/s]

DeepFool:   0%|          | 0/21 [00:00<?, ?it/s]

DeepFool:   0%|          | 0/21 [00:00<?, ?it/s]

In [ ]:
# Convert NumPy arrays to Tensor #
# Teacher MLP
x_test_deepfool_teacher_mlp_tensor = torch.tensor(x_test_deepfool_teacher_mlp, dtype=torch.float32)

# Student MLP
x_test_deepfool_student_mlp_tensor = torch.tensor(x_test_deepfool_student_mlp, dtype=torch.float32)

# New Student MLP
x_test_deepfool_new_student_mlp_tensor = torch.tensor(x_test_deepfool_new_student_mlp, dtype=torch.float32)

# Label (labels_test = pd.factorize(y_test)[0]) #
# Teacher & Student & New Student MLP
labels_test_deepfool_all_model_tensor = torch.tensor(labels_test, dtype=torch.long)

# Create Dataset #
# Teacher MLP
x_test_deepfool_teacher_mlp_data = TensorDataset(x_test_deepfool_teacher_mlp_tensor, labels_test_deepfool_all_model_tensor)

# Student MLP
x_test_deepfool_student_mlp_data = TensorDataset(x_test_deepfool_student_mlp_tensor, labels_test_deepfool_all_model_tensor)

# New Student MLP
x_test_deepfool_new_student_mlp_data = TensorDataset(x_test_deepfool_new_student_mlp_tensor, labels_test_deepfool_all_model_tensor)

# Create DataLoader for the adversarial data #
# Teacher MLP
x_test_deepfool_teacher_mlp_loader = DataLoader(x_test_deepfool_teacher_mlp_data, batch_size=128, shuffle=False)

# Student MLP
x_test_deepfool_student_mlp_loader = DataLoader(x_test_deepfool_student_mlp_data, batch_size=128, shuffle=False)

# New Student MLP
x_test_deepfool_new_student_mlp_loader = DataLoader(x_test_deepfool_new_student_mlp_data, batch_size=128, shuffle=False)

In [ ]:
# Metrics for Teacher after the Deepfool Attack
test_metrics_teacher_mlp_deepfool = test(teacher_mlp, x_test_deepfool_teacher_mlp_loader, device)

# Metrics for Student after the Deepfool Attack
test_metrics_student_mlp_deepfool = test(student_mlp, x_test_deepfool_student_mlp_loader, device)

# Metrics for Student (KD) after the Deepfool Attack
test_metrics_new_student_mlp_deepfool = test(new_student_mlp, x_test_deepfool_new_student_mlp_loader, device)

In [ ]:
from tabulate import tabulate

# Data
data = [
    ["Teacher MLP Metrics", f"{test_metrics_teacher_mlp[0]:.2f}%", f"{test_metrics_teacher_mlp[1]:.2f}%", f"{test_metrics_teacher_mlp[2]:.2f}%", f"{test_metrics_teacher_mlp[3]:.2f}%"],

    ["Student MLP Metrics", f"{test_metrics_student_mlp[0]:.2f}%", f"{test_metrics_student_mlp[1]:.2f}%", f"{test_metrics_student_mlp[2]:.2f}%", f"{test_metrics_student_mlp[3]:.2f}%"],

    ["KD Student MLP Metrics", f"{test_metrics_kd_student_mlp[0]:.2f}%", f"{test_metrics_kd_student_mlp[1]:.2f}%", f"{test_metrics_kd_student_mlp[2]:.2f}%", f"{test_metrics_kd_student_mlp[3]:.2f}%"],

    ["Teacher MLP Metrics (Deepfool)", f"{test_metrics_teacher_mlp_deepfool[0]:.2f}%", f"{test_metrics_teacher_mlp_deepfool[1]:.2f}%", f"{test_metrics_teacher_mlp_deepfool[2]:.2f}%", f"{test_metrics_teacher_mlp_deepfool[3]:.2f}%"],

    ["Student MLP Metrics (Deepfool)", f"{test_metrics_student_mlp_deepfool[0]:.2f}%", f"{test_metrics_student_mlp_deepfool[1]:.2f}%", f"{test_metrics_student_mlp_deepfool[2]:.2f}%", f"{test_metrics_student_mlp_deepfool[3]:.2f}%"],

    ["KD Student MLP Metrics (Deepfool)", f"{test_metrics_new_student_mlp_deepfool[0]:.2f}%", f"{test_metrics_new_student_mlp_deepfool[1]:.2f}%", f"{test_metrics_new_student_mlp_deepfool[2]:.2f}%", f"{test_metrics_new_student_mlp_deepfool[3]:.2f}%"],
]

# Column headers
headers = ["Metrics", "Accuracy", "Precision", "Recall", "F1-score"]

# Display table
print(tabulate(data, headers=headers, tablefmt="grid"))


+-----------------------------------+------------+-------------+----------+------------+
| Metrics                           | Accuracy   | Precision   | Recall   | F1-score   |
+===================================+============+=============+==========+============+
| Teacher MLP Metrics               | 98.90%     | 98.93%      | 98.90%   | 98.91%     |
+-----------------------------------+------------+-------------+----------+------------+
| Student MLP Metrics               | 98.77%     | 98.82%      | 98.77%   | 98.78%     |
+-----------------------------------+------------+-------------+----------+------------+
| KD Student MLP Metrics            | 98.81%     | 98.84%      | 98.81%   | 98.82%     |
+-----------------------------------+------------+-------------+----------+------------+
| Teacher MLP Metrics (Deepfool)    | 33.01%     | 31.23%      | 33.01%   | 30.61%     |
+-----------------------------------+------------+-------------+----------+------------+
| Student MLP Metrics

### **Carlini and Wagner L_2 Attack**

In [ ]:
from art.attacks.evasion import CarliniL2Method

# Define parameters for the Carlini & Wagner L2 Attack
carwal2_params = {
    'confidence': 0,  # Degree of confidence for the attack; 0 means no specific level required
    'targeted': False,  # Specifies if the attack is targeted; False means untargeted attack
    'learning_rate': 0.01,  # Learning rate for optimization; controls the magnitude of adjustments per step
    'max_iter': 50, # Maximum iterations for each sample; limits the attack duration per input
    'binary_search_steps': 3, # Number of binary search steps to optimize the 'const' parameter for the best results
    'initial_const': 0.01,  # Initial constant value for balancing the perturbation and attack success
    'batch_size': 128,  # Initial constant value for balancing the perturbation and attack success
}

# Create attackers for each classifier
# Teacher MLP
teacher_mlp_carwal2_attacker = CarliniL2Method(classifier=teacher_mlp_classifier, **carwal2_params)

# Student MLP
student_mlp_carwal2_attacker = CarliniL2Method(classifier=student_mlp_classifier, **carwal2_params)

# New Student MLP
new_student_mlp_carwal2_attacker = CarliniL2Method(classifier=new_student_mlp_classifier, **carwal2_params)

# Generate adversarial samples from the teacher model
x_test_carwal2_teacher_mlp = teacher_mlp_carwal2_attacker.generate(x_test_numpy)

# Generate adversarial samples from the original student model
x_test_carwal2_student_mlp = student_mlp_carwal2_attacker.generate(x_test_numpy)

# Generate adversarial samples from the new student model
x_test_carwal2_new_student_mlp = new_student_mlp_carwal2_attacker.generate(x_test_numpy)


C&W L_2:   0%|          | 0/21 [00:00<?, ?it/s]

C&W L_2:   0%|          | 0/21 [00:00<?, ?it/s]

C&W L_2:   0%|          | 0/21 [00:00<?, ?it/s]

In [ ]:
# Convert NumPy arrays to Tensor #
# Teacher MLP
x_test_carwal2_teacher_mlp_tensor = torch.tensor(x_test_carwal2_teacher_mlp, dtype=torch.float32)

# Student MLP
x_test_carwal2_student_mlp_tensor = torch.tensor(x_test_carwal2_student_mlp, dtype=torch.float32)

# New Student MLP
x_test_carwal2_new_student_mlp_tensor = torch.tensor(x_test_carwal2_new_student_mlp, dtype=torch.float32)

# Label (labels_test = pd.factorize(y_test)[0]) #
# Teacher & Student & New Student MLP
labels_test_carwal2_all_model_tensor = torch.tensor(labels_test, dtype=torch.long)

# Create Dataset #
# Teacher MLP
x_test_carwal2_teacher_mlp_data = TensorDataset(x_test_carwal2_teacher_mlp_tensor, labels_test_carwal2_all_model_tensor)

# Student MLP
x_test_carwal2_student_mlp_data = TensorDataset(x_test_carwal2_student_mlp_tensor, labels_test_carwal2_all_model_tensor)

# New Student MLP
x_test_carwal2_new_student_mlp_data = TensorDataset(x_test_carwal2_new_student_mlp_tensor, labels_test_carwal2_all_model_tensor)

# Create DataLoader for the adversarial data #
# Teacher MLP
x_test_carwal2_teacher_mlp_loader = DataLoader(x_test_carwal2_teacher_mlp_data, batch_size=128, shuffle=False)

# Student MLP
x_test_carwal2_student_mlp_loader = DataLoader(x_test_carwal2_student_mlp_data, batch_size=128, shuffle=False)

# New Student MLP
x_test_carwal2_new_student_mlp_loader = DataLoader(x_test_carwal2_new_student_mlp_data, batch_size=128, shuffle=False)

In [ ]:
# Metrics for Teacher after the Carlini and Wagner L_2 Attack
test_metrics_teacher_mlp_carwal2 = test(teacher_mlp, x_test_carwal2_teacher_mlp_loader, device)

# Metrics for Student after the Carlini and Wagner L_2 Attack
test_metrics_student_mlp_carwal2 = test(student_mlp, x_test_carwal2_student_mlp_loader, device)

# Metrics for Student (KD) after the Carlini and Wagner L_2 Attack
test_metrics_new_student_mlp_carwal2 = test(new_student_mlp, x_test_carwal2_new_student_mlp_loader, device)

In [ ]:
from tabulate import tabulate

# Data
data = [
    ["Teacher MLP Metrics", f"{test_metrics_teacher_mlp[0]:.2f}%", f"{test_metrics_teacher_mlp[1]:.2f}%", f"{test_metrics_teacher_mlp[2]:.2f}%", f"{test_metrics_teacher_mlp[3]:.2f}%"],

    ["Student MLP Metrics", f"{test_metrics_student_mlp[0]:.2f}%", f"{test_metrics_student_mlp[1]:.2f}%", f"{test_metrics_student_mlp[2]:.2f}%", f"{test_metrics_student_mlp[3]:.2f}%"],

    ["KD Student MLP Metrics", f"{test_metrics_kd_student_mlp[0]:.2f}%", f"{test_metrics_kd_student_mlp[1]:.2f}%", f"{test_metrics_kd_student_mlp[2]:.2f}%", f"{test_metrics_kd_student_mlp[3]:.2f}%"],

    ["Teacher MLP Metrics (Carlini and Wagner L_2)", f"{test_metrics_teacher_mlp_carwal2[0]:.2f}%", f"{test_metrics_teacher_mlp_carwal2[1]:.2f}%", f"{test_metrics_teacher_mlp_carwal2[2]:.2f}%", f"{test_metrics_teacher_mlp_carwal2[3]:.2f}%"],

    ["Student MLP Metrics (Carlini and Wagner L_2)", f"{test_metrics_student_mlp_carwal2[0]:.2f}%", f"{test_metrics_student_mlp_carwal2[1]:.2f}%", f"{test_metrics_student_mlp_carwal2[2]:.2f}%", f"{test_metrics_student_mlp_carwal2[3]:.2f}%"],

    ["KD Student MLP Metrics (Carlini and Wagner L_2)", f"{test_metrics_new_student_mlp_carwal2[0]:.2f}%", f"{test_metrics_new_student_mlp_carwal2[1]:.2f}%", f"{test_metrics_new_student_mlp_carwal2[2]:.2f}%", f"{test_metrics_new_student_mlp_carwal2[3]:.2f}%"],
]

# Column headers
headers = ["Metrics", "Accuracy", "Precision", "Recall", "F1-score"]

# Display table
print(tabulate(data, headers=headers, tablefmt="grid"))


+-------------------------------------------------+------------+-------------+----------+------------+
| Metrics                                         | Accuracy   | Precision   | Recall   | F1-score   |
+=================================================+============+=============+==========+============+
| Teacher MLP Metrics                             | 98.90%     | 98.93%      | 98.90%   | 98.91%     |
+-------------------------------------------------+------------+-------------+----------+------------+
| Student MLP Metrics                             | 98.77%     | 98.82%      | 98.77%   | 98.78%     |
+-------------------------------------------------+------------+-------------+----------+------------+
| KD Student MLP Metrics                          | 98.81%     | 98.84%      | 98.81%   | 98.82%     |
+-------------------------------------------------+------------+-------------+----------+------------+
| Teacher MLP Metrics (Carlini and Wagner L_2)    | 97.05%     | 97.14%  

### **Sign-OPT Attack**

In [ ]:
from art.attacks.evasion import SignOPTAttack

# Define parameters for the Sign-OPT Attack
sign_opt_params = {
    'targeted': False,  # Specifies if the attack is targeted; False means untargeted attack
    'epsilon': 0.01,  # Maximum perturbation allowed to generate adversarial examples
    'num_trial': 10,  # Number of trials to optimize gradient direction, improving attack accuracy
    'max_iter': 20, # Maximum iterations for each sample; limits attack duration per input
    'query_limit': 100, # Limits the number of queries to the target model to prevent prolonged attack
    'k': 10,  # Number of gradient directions evaluated per iteration to find the optimal one
    'alpha': 0.5, # Controls the gradient influence in the optimization process
    'beta': 0.001,  # Step size for perturbation adjustment in each iteration
    'batch_size': 128,  # Number of samples in each batch, facilitating batch processing
    'verbose': True, # When False, suppresses output details of the attack process
}

# Use the wrapped estimator to create attackers and generate adversarial samples #
# Teacher MLP
teacher_mlp_signopt_attacker = SignOPTAttack(estimator=teacher_mlp_classifier, **sign_opt_params)

# Student MLP
student_mlp_signopt_attacker = SignOPTAttack(estimator=student_mlp_classifier, **sign_opt_params)

# New Student MLP
new_student_mlp_signopt_attacker = SignOPTAttack(estimator=new_student_mlp_classifier, **sign_opt_params)

# Generate adversarial samples from the teacher model
x_test_signopt_teacher_mlp = teacher_mlp_signopt_attacker.generate(x_test_numpy)

# Generate adversarial samples from the original student model
x_test_signopt_student_mlp = student_mlp_signopt_attacker.generate(x_test_numpy)

# Generate adversarial samples from the new student model
x_test_signopt_new_student_mlp = new_student_mlp_signopt_attacker.generate(x_test_numpy)

Sign_OPT attack:   0%|          | 0/2614 [00:00<?, ?it/s]

Streaming output truncated to the last 5000 lines.
Succeed distortion 1.5936299333092618 org_label 2 predict_lable None                     queries 134 Line Search queries 86
Searching for the initial direction on 10 random directions: 
Found distortion 2.1824722492601722 with iteration/num_directions=2/10
query_count=135 > query_limit=100
Succeed distortion 1.6620205421150525 org_label 3 predict_lable None                     queries 135 Line Search queries 98
Searching for the initial direction on 10 random directions: 
Found distortion 8.02873132401146 with iteration/num_directions=0/10
Found distortion 4.234881353888369 with iteration/num_directions=3/10
Found distortion 4.089100526032348 with iteration/num_directions=7/10
Found distortion 3.540026969069997 with iteration/num_directions=8/10
query_count=231 > query_limit=100
Succeed distortion 2.134012537752895 org_label 3 predict_lable None                     queries 231 Line Search queries 157
Searching for the initial direction

Sign_OPT attack:   0%|          | 0/2614 [00:00<?, ?it/s]

Streaming output truncated to the last 5000 lines.
Succeed distortion 3.8333057915226965 org_label 0 predict_lable None                     queries 132 Line Search queries 98
Searching for the initial direction on 10 random directions: 
Couldn't find valid initial, failed
Searching for the initial direction on 10 random directions: 
Found distortion 4.420333626214415 with iteration/num_directions=4/10
query_count=184 > query_limit=100
Succeed distortion 2.624554339421438 org_label 2 predict_lable None                     queries 184 Line Search queries 151
Searching for the initial direction on 10 random directions: 
Couldn't find valid initial, failed
Searching for the initial direction on 10 random directions: 
Found distortion 2.797912167967297 with iteration/num_directions=4/10
Found distortion 0.768469528555471 with iteration/num_directions=6/10
Found distortion 0.37072651084609637 with iteration/num_directions=7/10
query_count=194 > query_limit=100
Succeed distortion 0.1932610055

Sign_OPT attack:   0%|          | 0/2614 [00:00<?, ?it/s]

Streaming output truncated to the last 5000 lines.
Searching for the initial direction on 10 random directions: 
Found distortion 7.384105581790209 with iteration/num_directions=9/10
query_count=178 > query_limit=100
Succeed distortion 4.841360725522977 org_label 0 predict_lable None                     queries 178 Line Search queries 144
Searching for the initial direction on 10 random directions: 
Found distortion 2.7636256739497185 with iteration/num_directions=0/10
Found distortion 1.277232275582719 with iteration/num_directions=8/10
query_count=173 > query_limit=100
Succeed distortion 0.7737577212646174 org_label 1 predict_lable None                     queries 173 Line Search queries 126
Searching for the initial direction on 10 random directions: 
Found distortion 0.772662270697765 with iteration/num_directions=2/10
query_count=136 > query_limit=100
Succeed distortion 0.5641381910001175 org_label 2 predict_lable None                     queries 136 Line Search queries 98
Searchi

In [ ]:
# Convert NumPy arrays to Tensor #
# Teacher MLP
x_test_signopt_teacher_mlp_tensor = torch.tensor(x_test_signopt_teacher_mlp, dtype=torch.float32)

# Student MLP
x_test_signopt_student_mlp_tensor = torch.tensor(x_test_signopt_student_mlp, dtype=torch.float32)

# New Student MLP
x_test_signopt_new_student_mlp_tensor = torch.tensor(x_test_signopt_new_student_mlp, dtype=torch.float32)

# Label (labels_test = pd.factorize(y_test)[0]) #
# Teacher & Student & New Student MLP
labels_test_signopt_all_model_tensor = torch.tensor(labels_test, dtype=torch.long)

# Create Dataset #
# Teacher MLP
x_test_signopt_teacher_mlp_data = TensorDataset(x_test_signopt_teacher_mlp_tensor, labels_test_signopt_all_model_tensor)

# Student MLP
x_test_signopt_student_mlp_data = TensorDataset(x_test_signopt_student_mlp_tensor, labels_test_signopt_all_model_tensor)

# New Student MLP
x_test_signopt_new_student_mlp_data = TensorDataset(x_test_signopt_new_student_mlp_tensor, labels_test_signopt_all_model_tensor)

# Create DataLoader for the adversarial data #
# Teacher MLP
x_test_signopt_teacher_mlp_loader = DataLoader(x_test_signopt_teacher_mlp_data, batch_size=128, shuffle=False)

# Student MLP
x_test_signopt_student_mlp_loader = DataLoader(x_test_signopt_student_mlp_data, batch_size=128, shuffle=False)

# New Student MLP
x_test_signopt_new_student_mlp_loader = DataLoader(x_test_signopt_new_student_mlp_data, batch_size=128, shuffle=False)

In [ ]:
# Metrics for Teacher after the Sign-OPT Attack
test_metrics_teacher_mlp_signopt = test(teacher_mlp, x_test_signopt_teacher_mlp_loader, device)

# Metrics for Student after the Sign-OPT Attack
test_metrics_student_mlp_signopt = test(student_mlp, x_test_signopt_student_mlp_loader, device)

# Metrics for Student (KD) after the Sign-OPT Attack
test_metrics_new_student_mlp_signopt = test(new_student_mlp, x_test_signopt_new_student_mlp_loader, device)

In [ ]:
from tabulate import tabulate

# Data
data = [
    ["Teacher MLP Metrics", f"{test_metrics_teacher_mlp[0]:.2f}%", f"{test_metrics_teacher_mlp[1]:.2f}%", f"{test_metrics_teacher_mlp[2]:.2f}%", f"{test_metrics_teacher_mlp[3]:.2f}%"],

    ["Student MLP Metrics", f"{test_metrics_student_mlp[0]:.2f}%", f"{test_metrics_student_mlp[1]:.2f}%", f"{test_metrics_student_mlp[2]:.2f}%", f"{test_metrics_student_mlp[3]:.2f}%"],

    ["KD Student MLP Metrics", f"{test_metrics_kd_student_mlp[0]:.2f}%", f"{test_metrics_kd_student_mlp[1]:.2f}%", f"{test_metrics_kd_student_mlp[2]:.2f}%", f"{test_metrics_kd_student_mlp[3]:.2f}%"],

    ["Teacher MLP Metrics (Sign-OPT)", f"{test_metrics_teacher_mlp_signopt[0]:.2f}%", f"{test_metrics_teacher_mlp_signopt[1]:.2f}%", f"{test_metrics_teacher_mlp_signopt[2]:.2f}%", f"{test_metrics_teacher_mlp_signopt[3]:.2f}%"],

    ["Student MLP Metrics (Sign-OPT)", f"{test_metrics_student_mlp_signopt[0]:.2f}%", f"{test_metrics_student_mlp_signopt[1]:.2f}%", f"{test_metrics_student_mlp_signopt[2]:.2f}%", f"{test_metrics_student_mlp_signopt[3]:.2f}%"],

    ["KD Student MLP Metrics (Sign-OPT)", f"{test_metrics_new_student_mlp_signopt[0]:.2f}%", f"{test_metrics_new_student_mlp_signopt[1]:.2f}%", f"{test_metrics_new_student_mlp_signopt[2]:.2f}%", f"{test_metrics_new_student_mlp_signopt[3]:.2f}%"],
]

# Column headers
headers = ["Metrics", "Accuracy", "Precision", "Recall", "F1-score"]

# Display table
print(tabulate(data, headers=headers, tablefmt="grid"))


+-----------------------------------+------------+-------------+----------+------------+
| Metrics                           | Accuracy   | Precision   | Recall   | F1-score   |
+===================================+============+=============+==========+============+
| Teacher MLP Metrics               | 98.90%     | 98.93%      | 98.90%   | 98.91%     |
+-----------------------------------+------------+-------------+----------+------------+
| Student MLP Metrics               | 98.77%     | 98.82%      | 98.77%   | 98.78%     |
+-----------------------------------+------------+-------------+----------+------------+
| KD Student MLP Metrics            | 98.81%     | 98.84%      | 98.81%   | 98.82%     |
+-----------------------------------+------------+-------------+----------+------------+
| Teacher MLP Metrics (Sign-OPT)    | 31.56%     | 44.56%      | 31.56%   | 35.75%     |
+-----------------------------------+------------+-------------+----------+------------+
| Student MLP Metrics

## **RESULT ART.ATTACKS.EVASION**

In [ ]:
from tabulate import tabulate

# Data
data_evasion = [
    ["Teacher - Student - KD Student"],
    ["Teacher MLP Metrics", f"{test_metrics_teacher_mlp[0]:.2f}%", f"{test_metrics_teacher_mlp[1]:.2f}%", f"{test_metrics_teacher_mlp[2]:.2f}%", f"{test_metrics_teacher_mlp[3]:.2f}%"],
    ["Student MLP Metrics", f"{test_metrics_student_mlp[0]:.2f}%", f"{test_metrics_student_mlp[1]:.2f}%", f"{test_metrics_student_mlp[2]:.2f}%", f"{test_metrics_student_mlp[3]:.2f}%"],
    ["KD Student MLP Metrics", f"{test_metrics_kd_student_mlp[0]:.2f}%", f"{test_metrics_kd_student_mlp[1]:.2f}%", f"{test_metrics_kd_student_mlp[2]:.2f}%", f"{test_metrics_kd_student_mlp[3]:.2f}%"],

    ["Adversarial Attacks - Evasion"],
    ["Teacher MLP Metrics (FGM)", f"{test_metrics_teacher_mlp_fgm[0]:.2f}%", f"{test_metrics_teacher_mlp_fgm[1]:.2f}%", f"{test_metrics_teacher_mlp_fgm[2]:.2f}%", f"{test_metrics_teacher_mlp_fgm[3]:.2f}%"],
    ["Student MLP Metrics (FGM)", f"{test_metrics_student_mlp_fgm[0]:.2f}%", f"{test_metrics_student_mlp_fgm[1]:.2f}%", f"{test_metrics_student_mlp_fgm[2]:.2f}%", f"{test_metrics_student_mlp_fgm[3]:.2f}%"],
    ["KD Student MLP Metrics (FGM)", f"{test_metrics_new_student_mlp_fgm[0]:.2f}%", f"{test_metrics_new_student_mlp_fgm[1]:.2f}%", f"{test_metrics_new_student_mlp_fgm[2]:.2f}%", f"{test_metrics_new_student_mlp_fgm[3]:.2f}%"],

    ["Teacher MLP Metrics (PGD)", f"{test_metrics_teacher_mlp_pgd[0]:.2f}%", f"{test_metrics_teacher_mlp_pgd[1]:.2f}%", f"{test_metrics_teacher_mlp_pgd[2]:.2f}%", f"{test_metrics_teacher_mlp_pgd[3]:.2f}%"],
    ["Student MLP Metrics (PGD)", f"{test_metrics_student_mlp_pgd[0]:.2f}%", f"{test_metrics_student_mlp_pgd[1]:.2f}%", f"{test_metrics_student_mlp_pgd[2]:.2f}%", f"{test_metrics_student_mlp_pgd[3]:.2f}%"],
    ["KD Student MLP Metrics (PGD)", f"{test_metrics_new_student_mlp_pgd[0]:.2f}%", f"{test_metrics_new_student_mlp_pgd[1]:.2f}%", f"{test_metrics_new_student_mlp_pgd[2]:.2f}%", f"{test_metrics_new_student_mlp_pgd[3]:.2f}%"],

    ["Teacher MLP Metrics (Deepfool)", f"{test_metrics_teacher_mlp_deepfool[0]:.2f}%", f"{test_metrics_teacher_mlp_deepfool[1]:.2f}%", f"{test_metrics_teacher_mlp_deepfool[2]:.2f}%", f"{test_metrics_teacher_mlp_deepfool[3]:.2f}%"],
    ["Student MLP Metrics (Deepfool)", f"{test_metrics_student_mlp_deepfool[0]:.2f}%", f"{test_metrics_student_mlp_deepfool[1]:.2f}%", f"{test_metrics_student_mlp_deepfool[2]:.2f}%", f"{test_metrics_student_mlp_deepfool[3]:.2f}%"],
    ["KD Student MLP Metrics (Deepfool)", f"{test_metrics_new_student_mlp_deepfool[0]:.2f}%", f"{test_metrics_new_student_mlp_deepfool[1]:.2f}%", f"{test_metrics_new_student_mlp_deepfool[2]:.2f}%", f"{test_metrics_new_student_mlp_deepfool[3]:.2f}%"],

    ["Teacher MLP Metrics (Carlini and Wagner L_2)", f"{test_metrics_teacher_mlp_carwal2[0]:.2f}%", f"{test_metrics_teacher_mlp_carwal2[1]:.2f}%", f"{test_metrics_teacher_mlp_carwal2[2]:.2f}%", f"{test_metrics_teacher_mlp_carwal2[3]:.2f}%"],
    ["Student MLP Metrics (Carlini and Wagner L_2)", f"{test_metrics_student_mlp_carwal2[0]:.2f}%", f"{test_metrics_student_mlp_carwal2[1]:.2f}%", f"{test_metrics_student_mlp_carwal2[2]:.2f}%", f"{test_metrics_student_mlp_carwal2[3]:.2f}%"],
    ["KD Student MLP Metrics (Carlini and Wagner L_2)", f"{test_metrics_new_student_mlp_carwal2[0]:.2f}%", f"{test_metrics_new_student_mlp_carwal2[1]:.2f}%", f"{test_metrics_new_student_mlp_carwal2[2]:.2f}%", f"{test_metrics_new_student_mlp_carwal2[3]:.2f}%"],

    ["Teacher MLP Metrics (Sign-OPT)", f"{test_metrics_teacher_mlp_signopt[0]:.2f}%", f"{test_metrics_teacher_mlp_signopt[1]:.2f}%", f"{test_metrics_teacher_mlp_signopt[2]:.2f}%", f"{test_metrics_teacher_mlp_signopt[3]:.2f}%"],
    ["Student MLP Metrics (Sign-OPT)", f"{test_metrics_student_mlp_signopt[0]:.2f}%", f"{test_metrics_student_mlp_signopt[1]:.2f}%", f"{test_metrics_student_mlp_signopt[2]:.2f}%", f"{test_metrics_student_mlp_signopt[3]:.2f}%"],
    ["KD Student MLP Metrics (Sign-OPT)", f"{test_metrics_new_student_mlp_signopt[0]:.2f}%", f"{test_metrics_new_student_mlp_signopt[1]:.2f}%", f"{test_metrics_new_student_mlp_signopt[2]:.2f}%", f"{test_metrics_new_student_mlp_signopt[3]:.2f}%"],

]

# Column headers
headers = ["Metrics", "Accuracy", "Precision", "Recall", "F1-score"]

# Display table
print(tabulate(data_evasion, headers=headers, tablefmt="grid"))


+-------------------------------------------------+------------+-------------+----------+------------+
| Metrics                                         | Accuracy   | Precision   | Recall   | F1-score   |
+=================================================+============+=============+==========+============+
| Teacher - Student - KD Student                  |            |             |          |            |
+-------------------------------------------------+------------+-------------+----------+------------+
| Teacher MLP Metrics                             | 98.90%     | 98.93%      | 98.90%   | 98.91%     |
+-------------------------------------------------+------------+-------------+----------+------------+
| Student MLP Metrics                             | 98.77%     | 98.82%      | 98.77%   | 98.78%     |
+-------------------------------------------------+------------+-------------+----------+------------+
| KD Student MLP Metrics                          | 98.81%     | 98.84%  

In [ ]:
# Convert data into a DataFrame (result_evasion)
result_evasion = pd.DataFrame(data_evasion, columns=headers)

# Export to a CSV file
%cd /content/drive/MyDrive/KnowledgeDistillation/
result_evasion.to_csv("result_kd_aa_evasion.csv", index=False)

# Export to an Excel file
result_evasion.to_excel("result_kd_aa_evasion.xlsx", index=False)

/content/drive/MyDrive/KnowledgeDistillation
